## Data Cleaning Summary

This notebook will explain how the data cleaning is done in the model. Model data cleaning code can be found in the `model/data_processing` directory.

### Drop N/A Values from these columns
- delinq_2yrs
- days_since_earliest_cr_line
- inq_last_6mths
- open_acc
- pub_rec
- total_acc
- revol_util
- purpose
- annual_inc
- emp_length


### Drop Columns
- collections_12_mths_ex_med
    - All values are either 0 or N/A
- pymnt_plan, 
- initial_list_status
    - Huge imbalances, will likely lead to overfitting. Double check.
- mths_since_last_record
    - Large number of N/A values, not immediately clear how to impute. Noted for a later version of the model.
- mths_since_last_delinq
    - Large number of N/A values, not immediately clear how to impute. Noted for a later version of the model.
- zip_code
    - Not informative in model testing, removing for V1.
- addr_state
    - The data in this column is just to sparse for V1.
- emp_title
    - Will not make it into V1.

    
### Drop Outliers/Nonsensical Data
- revol_bal == 1207359, this is an outlier value.
- revol_util > 100

### Simplify/Cleanup Categorical Data
- emp_title
    - This can be dramatically cleaned up, but is going to be dropped from V1
- home_ownership, verification_status
    - Simplify

### Distill Commentary (extra)
- Notes
    - Will likely drop for V1.
- purpose
    - Will likely drop and use purpose_cat instead.
- purpose_cat
    - Will choose cutoff count (<100) and map all those values to an 'other' bucket.


## Final Transforms
- Dummies will be created for categorical values.

In [1]:
import datetime
import numpy as np
import pandas as pd
import pickle

df = pd.read_csv('../data/DR_Demo_Lending_Club.csv')
orig_df_length = len(df)

In [ ]:
## Create new variable from earliest_cr_line(datetime) = 
## current_date - earliest_cr_line = days since earliest credit line
current_date = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['days_since_earliest_cr_line'] = (current_date - df['earliest_cr_line']).dt.days
df = df.drop('earliest_cr_line', axis=1)

In [ ]:
## Convert emp_length to float64 from str.
df.loc[df['emp_length'] == 'na', 'emp_length'] = np.nan
df['emp_length'] = df['emp_length'].astype('float64')

In [ ]:
## Drop NA Values from the following columns
drop_na_value_columns = ['delinq_2yrs',
'days_since_earliest_cr_line',
'inq_last_6mths',
'open_acc',
'pub_rec',
'total_acc',
'revol_util',
'purpose',
'annual_inc']

df = df.dropna(axis=0, subset=drop_na_value_columns)
print('Rows dropped: {}'.format(orig_df_length - len(df)))
print('New data length: {}'.format(len(df)))

In [ ]:
## Dropping the following for V1 of the model
df = df.drop(['collections_12_mths_ex_med', 
             'pymnt_plan', 
             'initial_list_status',
             'mths_since_last_record',
             'mths_since_last_delinq',
             'zip_code',
             'addr_state'], axis=1)

In [ ]:
## Dropping very specific outliers/non-sensical rows
df = df[df['revol_util'] <= 100]

In [ ]:
## Creating mapping to consolidate 'VERIFIED - income' and 'VERIFIED - income source' as simply ''VERIFIED - income'
value_map = {
    'VERIFIED - income': 'VERIFIED - income',
    'VERIFIED - income source': 'VERIFIED - income',
    'not verified': 'not verified'
}
df['verification_status'] = [value_map[x] for x in df['verification_status']]

In [ ]:
## Mapping lower represented groups to an 'other' bucket
purpose_cat_count = df.groupby(['purpose_cat']).count().sort_values('Id', ascending=False)['Id']
valid_values = list(purpose_cat_count[purpose_cat_count > 100].index)
df['purpose_cat'] = [purpose if purpose in valid_values else 'other' for purpose in df['purpose_cat']]


In [ ]:
valid_values

In [ ]:
## Dropping following text columns for V1
df = df.drop(['emp_title', 'Notes','purpose'], axis=1)

In [ ]:
## Splitting out categorical data to dummies:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = df.select_dtypes(exclude=numerics)

for col in categorical_columns:
    dummies = pd.get_dummies(categorical_columns[col])
    df = df.drop(col, axis=1)
    df = pd.concat([df, dummies], axis=1)
    print(col)

In [ ]:
## Final df:
df.head()

In [ ]:
from library_code import clean_and_transform_data

In [ ]:
df = pd.read_csv('../data/DR_Demo_Lending_Club.csv')
# df = clean_and_transform_data(df)

In [ ]:
df.iloc[0:4]